# SVD image compression demo

Лабораторная работа по сжатию изображений с потерями с помощью сингулярного разложения. Используем `numpy.linalg.svd` для разных чисел сингулярных значений и сравниваем визуальное качество и требуемые байты.


In [ ]:
import os
from pathlib import Path
import urllib.request

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Гарантируем повторяемость
np.random.seed(0)

IMG_URL = (
    "https://upload.wikimedia.org/wikipedia/commons/5/5e/King%27s_College_Chapel_and_Gibbs%27_Building_at_sunset._Cambridge_UK.jpg"
)
IMG_PATH = Path("cambridge.jpg")
RESIZE_FACTOR = 0.5  # при необходимости можно изменить

# Скачиваем картинку, если её нет рядом с ноутбуком
if not IMG_PATH.exists():
    IMG_PATH.parent.mkdir(parents=True, exist_ok=True)
    print(f"Downloading image to {IMG_PATH} ...")
    urllib.request.urlretrieve(IMG_URL, IMG_PATH)
    print("Done.")
else:
    print(f"Using existing image: {IMG_PATH}")


In [ ]:
# Чтение изображения и лёгкое уменьшение для ускорения SVD
img = Image.open(IMG_PATH).convert("RGB")
orig_w, orig_h = img.size
if RESIZE_FACTOR != 1:
    img = img.resize((int(orig_w * RESIZE_FACTOR), int(orig_h * RESIZE_FACTOR)))

x = np.array(img, dtype=np.float32)
print(f"Original size (file): {IMG_PATH.stat().st_size/1024:.1f} KB")
print(f"Array shape: {x.shape} (H, W, C)")

plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis("off")
plt.title("Исходное изображение")
plt.show()


In [ ]:
def compress_svd_channel(channel: np.ndarray, k: int) -> np.ndarray:
    """SVD-аппроксимация одного канала матрицы."""
    U, S, Vt = np.linalg.svd(channel, full_matrices=False)
    U_k = U[:, :k]
    S_k = np.diag(S[:k])
    Vt_k = Vt[:k, :]
    return U_k @ S_k @ Vt_k


def compress_image_color(x: np.ndarray, k: int) -> np.ndarray:
    """Применяем SVD к каждому каналу RGB отдельно."""
    comps = []
    for c in range(3):
        comps.append(compress_svd_channel(x[:, :, c], k))
    recon = np.stack(comps, axis=2)
    return np.clip(recon, 0, 255)


def compress_image_gray(x: np.ndarray, k: int) -> np.ndarray:
    gray = x.mean(axis=2)
    recon = compress_svd_channel(gray, k)
    recon = np.clip(recon, 0, 255)
    return np.repeat(recon[:, :, None], 3, axis=2)


def to_uint8_image(arr: np.ndarray) -> Image.Image:
    return Image.fromarray(arr.astype(np.uint8))



In [ ]:
k_values = [1, 2, 10, 30, 100]

fig, axes = plt.subplots(1, len(k_values), figsize=(3 * len(k_values), 3))
for ax, k in zip(axes, k_values):
    recon = compress_image_color(x, k)
    ax.imshow(recon.astype(np.uint8))
    ax.set_title(f"k = {k}")
    ax.axis("off")

plt.suptitle("Цветное изображение, SVD по каждому каналу")
plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(1, len(k_values), figsize=(3 * len(k_values), 3))
for ax, k in zip(axes, k_values):
    recon = compress_image_gray(x, k)
    ax.imshow(recon.astype(np.uint8), cmap="gray")
    ax.set_title(f"gray k = {k}")
    ax.axis("off")

plt.suptitle("Одноканальное приближение (усреднение RGB)")
plt.tight_layout()
plt.show()


In [ ]:
output_dir = Path("compressed")
output_dir.mkdir(exist_ok=True)

h, w, _ = x.shape
orig_file_size = IMG_PATH.stat().st_size
print(f"Размер исходного файла: {orig_file_size/1024:.1f} KB")

rows = []
for k in k_values:
    recon = compress_image_color(x, k)
    out_path = output_dir / f"cambridge_k{k}.png"
    to_uint8_image(recon).save(out_path)
    file_size = out_path.stat().st_size

    # Теоретический вес хранения усеченного SVD (float32):
    # для каждой матрицы: U (h x k), S (k), V (k x w)
    svd_params = (h * k + k + k * w) * 3  # три канала
    svd_bytes = svd_params * 4  # float32 -> 4 байта

    rows.append((k, file_size, svd_bytes))

for k, fs, svd_bytes in rows:
    print(
        f"k={k:3d}: файл {fs/1024:.1f} KB | усечённый SVD ~{svd_bytes/1024:.1f} KB"
    )

print("\nСжатые изображения сохранены в папку 'compressed/'.")
